# Utilisation de l'api SNCF

Ce notebook a pour but d'utiliser l'api SNCF afin de répondre a la question suivante : 

Dans quelle gare il y a le plus de retard de train en France.

Pour utiliser l'api nous avons besoin d'une clef API qui sera stocker dans un secret.


__Librairies :__

python-dotenv

In [2]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
from datetime import datetime

Je charge ma clé API dans une variable

In [3]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [4]:
def requete_api(requete):
    reponse = requests.get(requete, auth=(api_key,""))
    reponse.status_code
    reponse_json = reponse.json()
    return reponse_json

In [5]:
def ecrire_json(data,nom_fichier):
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [6]:
def main():
    pass

### Exemples

__Le détail des modes de transport couvert par l’API__

In [21]:
requete = "https://api.sncf.com/v1/coverage/sncf/commercial_modes"
reponse = requete_api(requete)

Mon fichier JSON est organisé en plusieur clés. Je récupère toutes les clés.

In [17]:
for key in reponse.keys():
    print(key)

pagination
feed_publishers
disruptions
context
commercial_modes
links


Les données qui m'interesse sont stocké dans la clé commercial_modes, je récupère donc les données de cette clé.

Je peux ensuite les charger dans un dataframe pandas

In [22]:
data = reponse.get("commercial_modes")
dataframe = pd.DataFrame(data)
dataframe.head()

,id,name
0,commercial_mode:DBS,DB SNCF
1,commercial_mode:EUROSTAR,Eurostar
2,commercial_mode:FR:Branding::45d07532-11f7-483...,SOLEA
3,commercial_mode:FR:Branding::5c5f14b9-bff2-4f9...,TER NA
4,commercial_mode:FR:Branding::6dfb1ec4-50b5-42a...,TER HDF


__Itinéraires entre Paris et Lyon__

In [24]:
requete = "https://api.sncf.com/v1/coverage/sncf/journeys?from=admin:fr:75056&to=admin:fr:69123&datetime=20240123T105527"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")
for key in reponse.keys():
    print(key)

feed_publishers
links
journeys
tickets
disruptions
terminus
context
notes
exceptions


### Exercice

Récupérer les informations sur le trajet entre Paris Gare de Lyon et Lyon Perrache le 17 novembre à 19h57

* Indice : utiliser la requête “journeys”

* Autre indice : le format de la date est AAAAMMJJTHHMMSS (Année, mois, jour, heure, minutes, secondes

Combien y a t-il d'arrêts entre ces deux gares ?

Combien de temps d'arrêt à chacune d'elles ? 

__Je recherche les stop_area :__

In [20]:
#Dans la région sncf je recherche les places contenant "lyon"
requete = "https://api.sncf.com/v1/coverage/sncf/places?q=lyon"
reponse = requete_api(requete)
ecrire_json(reponse,"liste place.json")

In [31]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
code_gare_lyon_perrache = "stop_area:SNCF:87722025"
date = "20240124T195700" 
url_base = "https://api.sncf.com/v1/coverage/sncf/journeys?"


In [39]:
requete = f"{url_base}from={code_gare_paris_gare_de_lyon}&to={code_gare_lyon_perrache}&datetime={date}"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")

1


In [41]:
#Nombre de voyage proposé
voyages = reponse.get("journeys")
nb_voyage = len(voyages)
print(f"il y a {nb_voyage} proposé.s")

il y a 2 proposé.s


Je veux garder uniquement le voyage le plus rapide

In [7]:
def voyage_rapide(journeys):
    '''Cette fonction parcours une liste de voyage et renvoie le voyage le plus rapide'''

    voyage_le_plus_rapide = None
    for voyage in voyages:
        if voyage["type"] == "fastest":
            voyage_le_plus_rapide = voyage
            break

    if voyage_le_plus_rapide is None:
        print("Pas de voyage fastest trouvé")
    else:
        print("voyage rapide trouvé")

    return voyage_le_plus_rapide

In [8]:
def decode_datetime(date_codee):
    #On enleve le T du code
    date_decodee = datetime.strptime(date_codee,"%Y%m%dT%H%M%S")
    return date_decodee

In [9]:
def liste_arrets(voyage):
    '''Cette fonction extrait les arrets d'un voyage et les présente dans un dataframe avec les heures de départ et d'arrivée'''

    nb_correspondance = voyage["nb_transfers"]
    print(f"Dans ce voyage il y a {nb_correspondance} correspondances")
    # Pour un voyage sans correspondances les sections sont organisé de la façon suivante :
    # type = crow_fly / type=public_transport / type = crow_fly
    # On garde la deuxième section
    # Les arrêts sont stocké sous la clé stop_date_times
    arrets = voyage["sections"][1]["stop_date_times"]
    nb_arrets = len(arrets)
    print(f"Dans ce voyage il y a {nb_arrets} arrets")

    #Je récupère mes données sous forme d'une liste
    donnees = []

    for arret in arrets:
        #J'ajoute a chaque element de ma liste un dictionnaire contenant des informations sur les arrêts
        donnees.append(dict(nom = arret["stop_point"]["name"],
                        arrive = decode_datetime(arret["arrival_date_time"]),
                        depart = decode_datetime(arret["departure_date_time"])))

    dataframe = pd.DataFrame(donnees)

    return dataframe


In [80]:
voyage = voyage_rapide(voyages)
arrets = liste_arrets(voyage)
arrets.head()

voyage rapide trouvé
Dans ce voyage il y a 0 correspondances
Dans ce voyage il y a 5 arrets


,nom,arrive,depart
0,Paris - Gare de Lyon - Hall 1 & 2,2024-01-24 21:00:00,2024-01-24 21:00:00
1,Le Creusot - Montceau-les-Mines - Montchanin TGV,2024-01-24 22:17:00,2024-01-24 22:20:00
2,Mâcon Loché TGV,2024-01-24 22:38:00,2024-01-24 22:41:00
3,Lyon Part Dieu,2024-01-24 23:10:00,2024-01-24 23:16:00
4,Lyon Perrache,2024-01-24 23:24:00,2024-01-24 23:24:00


__Question 2 :__

Vous êtes un peu pressé et vous avez peur de vous tromper en arrivant à la gare car d’autres TGV partent à peu près en même temps (à partir de 19h00) de la gare de Lyon.

* Si vous demandez à l’API, combien de résultats vous donne-t-elle ?

* Quels sont les horaires de départ de ces trains ?

* Parmi ces trains, combien de trains ont pour destination finale Lyon et qui partent le 17 novembre ?

Je veux tous les trains qui partent de gare de Lyon à 19h

In [161]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
date = "20240125T190000"
url_base = "https://api.sncf.com/v1/coverage/sncf"
network = "network:SNCF:OUI"
physical_mode = "physical_mode:LongDistanceTrain"

In [162]:
requete_network = f"{url_base}/stop_areas/{code_gare_paris_gare_de_lyon}/networks/{network}/departures?from_datetime={date}"
requete_physical_mode =f"{url_base}/stop_areas/{code_gare_paris_gare_de_lyon}/physical_modes/{physical_mode}/departures?from_datetime={date}"
reponse = requete_api(requete_physical_mode)
ecrire_json(reponse,"exercice_2.json")

In [163]:
data=[]
departs = reponse["departures"]

for depart in departs:
    
    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"])
    heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"])


    data.append(dict(commercial_mode = depart["display_informations"]["commercial_mode"],
                    reseau = depart["display_informations"]["network"],
                    numero = depart["display_informations"]["headsign"],
                    direction = depart["display_informations"]["direction"],
                    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"]),
                    heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"]),
                    delais=  heure_depart - heure_depart_prevue ))

dataframe = pd.DataFrame(data)
dataframe.head(20)

,commercial_mode,reseau,numero,direction,heure_depart,heure_depart_prevue,delais
0,TGV INOUI,TGV INOUI,6689,Saint-Étienne Châteaucreux (Saint-Étienne),2024-01-25 19:00:00,2024-01-25 19:00:00,0 days 00:00:00
1,TGV INOUI,TGV INOUI,6711,Mulhouse (Mulhouse),2024-01-25 19:09:00,2024-01-25 18:54:00,0 days 00:15:00
2,TGV INOUI,TGV INOUI,6927,Grenoble (Grenoble),2024-01-25 19:14:00,2024-01-25 19:14:00,0 days 00:00:00
3,OUIGO,OUIGO,7805,Lyon Perrache (Lyon),2024-01-25 19:26:00,2024-01-25 19:26:00,0 days 00:00:00
4,TGV INOUI,TGV INOUI,6129,Aix-en-Provence TGV (Aix-en-Provence),2024-01-25 19:56:00,2024-01-25 19:56:00,0 days 00:00:00
5,TGV INOUI,TGV INOUI,6227,Montpellier Saint-Roch (Montpellier),2024-01-25 20:09:00,2024-01-25 20:09:00,0 days 00:00:00
6,OUIGO,OUIGO,7827,Aix-en-Provence TGV (Aix-en-Provence),2024-01-25 20:13:00,2024-01-25 20:13:00,0 days 00:00:00
7,TGV Lyria,TGV Lyria,9789,Genève - Cornavin - CFF (Genève),2024-01-25 20:18:00,2024-01-25 20:18:00,0 days 00:00:00
8,TGV INOUI,TGV INOUI,6715,Mulhouse (Mulhouse),2024-01-25 20:22:00,2024-01-25 20:22:00,0 days 00:00:00
9,TGV INOUI,TGV INOUI,6633,Lyon Perrache (Lyon),2024-01-25 21:00:00,2024-01-25 21:00:00,0 days 00:00:00


__Question 3:__

En fait, vous n’êtes plus très sûr de vouloir aller à Lyon. Mais bon maintenant vous êtes Gare de Lyon et il est 18h00.

* Combien de tgv partent entre 18h00 et 20h00 ?

* Lequel arrive le plus tôt à sa destination finale ?

In [10]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
date_debut = "20240125T180000"
date_fin = "20240125T200000"
url_base = "https://api.sncf.com/v1/coverage/sncf"
physical_mode = "physical_mode:LongDistanceTrain"

Je fais ma requete en filtrant sur le physical_mode et sur les datetime. J'augmente également le nombre de réponse à 20

In [11]:
requete_physical_mode =f"{url_base}/stop_areas/{code_gare_paris_gare_de_lyon}\
/physical_modes/{physical_mode}/departures?from_datetime={date_debut}&max_datetime={date_fin}&count=20&"
reponse = requete_api(requete_physical_mode)
ecrire_json(reponse,"exercice_3.json")

In [18]:
data=[]
departs = reponse["departures"]

for depart in departs:
    
    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"])
    heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"])


    data.append(dict(commercial_mode = depart["display_informations"]["commercial_mode"],
                    reseau = depart["display_informations"]["network"],
                    numero = depart["display_informations"]["headsign"],
                    direction = depart["display_informations"]["direction"],
                    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"]),
                    heure_depart_prevue = decode_datetime(depart["stop_date_time"]["base_departure_date_time"]),
                    delais=  heure_depart - heure_depart_prevue,
                    heure_arrivee = decode_datetime(depart["stop_date_time"]["arrival_date_time"])))

dataframe = pd.DataFrame(data)
dataframe.head(20)

,commercial_mode,reseau,numero,direction,heure_depart,heure_depart_prevue,delais,heure_arrivee
0,TGV INOUI,TGV INOUI,6923,Grenoble (Grenoble),2024-01-25 18:09:00,2024-01-25 17:14:00,0 days 00:55:00,2024-01-25 18:09:00
1,TGV INOUI,TGV INOUI,6195,Miramas (Miramas),2024-01-25 18:09:00,2024-01-25 17:14:00,0 days 00:55:00,2024-01-25 18:09:00
2,TGV INOUI,TGV INOUI,6153,Toulon (Toulon),2024-01-25 18:10:00,2024-01-25 18:10:00,0 days 00:00:00,2024-01-25 18:10:00
3,TGV Lyria,TGV Lyria,9781,Genève - Cornavin - CFF (Genève),2024-01-25 18:18:00,2024-01-25 18:18:00,0 days 00:00:00,2024-01-25 18:18:00
4,TGV Lyria,TGV Lyria,9223,Zürich - Hauptbahnhof (Zürich),2024-01-25 18:22:00,2024-01-25 18:22:00,0 days 00:00:00,2024-01-25 18:22:00
5,TGV INOUI,TGV INOUI,6669,Lyon Perrache (Lyon),2024-01-25 18:26:00,2024-01-25 18:26:00,0 days 00:00:00,2024-01-25 18:26:00
6,TGV INOUI,TGV INOUI,6925,Grenoble (Grenoble),2024-01-25 18:34:00,2024-01-25 18:14:00,0 days 00:20:00,2024-01-25 18:34:00
7,TGV INOUI,TGV INOUI,6127,Marseille Saint-Charles (Marseille),2024-01-25 18:38:00,2024-01-25 18:38:00,0 days 00:00:00,2024-01-25 18:38:00
8,TGV INOUI,TGV INOUI,6665,Lyon Perrache (Lyon),2024-01-25 18:42:00,2024-01-25 18:42:00,0 days 00:00:00,2024-01-25 18:42:00
9,TGV INOUI,TGV INOUI,6951,Annecy (Annecy),2024-01-25 18:46:00,2024-01-25 18:46:00,0 days 00:00:00,2024-01-25 18:46:00


On utilise pandas pour filtrer le dataframe et gardes train qui arrivent avant 20h

In [19]:
limite_heure = "2024-01-25 20:00:00"
depart_avant_20h = dataframe.query("heure_depart < '2024-01-25 20:00:00'")
depart_avant_20h.tail()

,commercial_mode,reseau,numero,direction,heure_depart,heure_depart_prevue,delais,heure_arrivee
11,TGV INOUI,TGV INOUI,6689,Saint-Étienne Châteaucreux (Saint-Étienne),2024-01-25 19:00:00,2024-01-25 19:00:00,0 days 00:00:00,2024-01-25 19:00:00
12,TGV INOUI,TGV INOUI,6711,Mulhouse (Mulhouse),2024-01-25 19:09:00,2024-01-25 18:54:00,0 days 00:15:00,2024-01-25 19:09:00
13,TGV INOUI,TGV INOUI,6927,Grenoble (Grenoble),2024-01-25 19:14:00,2024-01-25 19:14:00,0 days 00:00:00,2024-01-25 19:14:00
14,OUIGO,OUIGO,7805,Lyon Perrache (Lyon),2024-01-25 19:26:00,2024-01-25 19:26:00,0 days 00:00:00,2024-01-25 19:26:00
15,TGV INOUI,TGV INOUI,6129,Aix-en-Provence TGV (Aix-en-Provence),2024-01-25 19:56:00,2024-01-25 19:56:00,0 days 00:00:00,2024-01-25 19:56:00


In [17]:
print(f"il y a {len(depart_avant_20h)} départs entre 18h et 20h")

il y a 16 départs entre 18h et 20h


In [23]:
print("Le train qui arrive en premier est le suivant :")
depart_avant_20h.sort_values(by=["heure_arrivee"]).head(1)

Le train qui arrive en premier est le suivant :


,commercial_mode,reseau,numero,direction,heure_depart,heure_depart_prevue,delais,heure_arrivee
0,TGV INOUI,TGV INOUI,6923,Grenoble (Grenoble),2024-01-25 18:09:00,2024-01-25 17:14:00,0 days 00:55:00,2024-01-25 18:09:00


__Question 4:__

On va essayer de voir jusqu’où on peut aller, en prenant des trains au départ de la Gare de Lyon :

* Quelles sont toutes les gares atteignables en partant le 17 novembre, sans faire de changement et sans partir après minuit ?

* Si on prend un de ces trains, jusqu’où peut-on aller, avec une correspondance, sans partir après 8h le lendemain matin ?

In [ ]:
def tous_trajet(gare,heure_min,heure_max):

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    physical_mode = "physical_mode:LongDistanceTrain"

    tous_les_departs = []
    heure_dernier_depart = heure_min
    heure_dernier_depart_decode = decode_datetime(heure_dernier_depart)
    heure_max_decode = decode_datetime(heure_max)


    while heure_dernier_depart_decode < heure_max_decode:

        requete =f"{base_url}/stop_areas/{gare}\
        /physical_modes/{physical_mode}/departures?from_datetime={heure_dernier_depart}"
        reponse = requete_api(requete)
        departs = reponse["departures"]
        dernier_depart = departs[-1]
        heure_dernier_depart = dernier_depart["stop_date_time"]["base_departure_date_time"]
        heure_dernier_depart_decode = decode_datetime(heure_dernier_depart)
        #J'ajoute la liste des depart a ma liste de tous les depart
        # Je n'utilise pas apprend car sinon ajoute une liste dans une liste
        tous_les_departs += depart
